In [170]:
# from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pymongo
# import datetime
# import time
import pandas as pd
import numpy as np
# import re
import warnings
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import string
%matplotlib inline
import matplotlib.pyplot as plt

In [9]:
mc = pymongo.MongoClient()
db = mc['chordify']
raw_html = db['raw_html']
parsed_songs_db = db["parsed_songs"]

In [10]:
html_docs = list(raw_html.find())

In [226]:
raw_html.find().count()

6456

In [82]:
parsed_songs = list(parsed_songs_db.find())

In [153]:
a = parsed_songs[4444]['_id']

In [155]:
str(a)

'5afde0942a1a05fe393db76d'

In [152]:
parsed_songs[4444]

{'_id': ObjectId('5afde0942a1a05fe393db76d'),
 'chord_idxs': [[0, 'G'],
  [6, 'D'],
  [12, 'Em'],
  [19, 'C'],
  [25, 'G'],
  [30, 'D'],
  [36, 'Em'],
  [41, 'C'],
  [45, 'G'],
  [50, 'D'],
  [54, 'Em'],
  [56, 'C'],
  [62, 'G'],
  [68, 'D'],
  [75, 'Em'],
  [77, 'C'],
  [83, 'G'],
  [89, 'D'],
  [95, 'Em'],
  [98, 'G'],
  [105, 'D'],
  [111, 'Em'],
  [115, 'C'],
  [121, 'G'],
  [123, 'D'],
  [127, 'Em'],
  [132, 'C'],
  [137, 'G'],
  [145, 'G'],
  [151, 'D'],
  [158, 'Em'],
  [165, 'C'],
  [171, 'G'],
  [176, 'D'],
  [181, 'Em'],
  [185, 'C'],
  [194, 'G'],
  [197, 'D'],
  [204, 'Em'],
  [209, 'C'],
  [215, 'G'],
  [219, 'D'],
  [228, 'Em'],
  [233, 'C'],
  [237, 'G'],
  [243, 'D'],
  [249, 'Em'],
  [252, 'G'],
  [259, 'D'],
  [265, 'Em'],
  [269, 'C'],
  [275, 'G'],
  [277, 'D'],
  [281, 'Em'],
  [286, 'C'],
  [291, 'G'],
  [299, 'G'],
  [306, 'D'],
  [311, 'Em'],
  [317, 'C'],
  [319, 'G'],
  [326, 'D'],
  [331, 'Em'],
  [338, 'C'],
  [342, 'G'],
  [348, 'D'],
  [354, 'Em'],
  [357,

In [80]:
parsed_songs_db.find().count()

5069

In [158]:
def get_phrase_for_chord(one_parsed_song):

    song_id = str(one_parsed_song['_id'])
    word_list = one_parsed_song['words']

    one_song_chord_phrase_tuples = []

    for chord_set in one_parsed_song['chord_idxs']:
        word_idx = chord_set[0]
        chord_name = chord_set[1]
        phrase = ' '.join(word_list[(word_idx - 8):(word_idx + 3)])
        one_song_chord_phrase_tuples.append((chord_name, phrase, song_id))

    return one_song_chord_phrase_tuples


def get_all_phrase_chord_tuples(parsed_songs):

    chord_phrase_tuples = []

    for song in parsed_songs:
        one_song_chord_phrase_tuples = get_phrase_for_chord(song)
        chord_phrase_tuples.extend(one_song_chord_phrase_tuples)

    return chord_phrase_tuples

In [159]:
pct = get_all_phrase_chord_tuples(parsed_songs[456:459])

In [160]:
pct

[('A', '', '5afddfd32a1a05fe393d96b7'),
 ('E', '', '5afddfd32a1a05fe393d96b7'),
 ('D',
  'in your face And the whole world is on your case',
  '5afddfd32a1a05fe393d96b7'),
 ('A',
  'on your case I could offer you a warm embrace To',
  '5afddfd32a1a05fe393d96b7'),
 ('D',
  'offer you a warm embrace To make you feel my love',
  '5afddfd32a1a05fe393d96b7'),
 ('A',
  'warm embrace To make you feel my love When the evening',
  '5afddfd32a1a05fe393d96b7'),
 ('E',
  'my love When the evening shadows and the stars appear And',
  '5afddfd32a1a05fe393d96b7'),
 ('D',
  'evening shadows and the stars appear And there is no one',
  '5afddfd32a1a05fe393d96b7'),
 ('Dm',
  'stars appear And there is no one there to dry your',
  '5afddfd32a1a05fe393d96b7'),
 ('A',
  'your tears I could hold you for a million years To',
  '5afddfd32a1a05fe393d96b7'),
 ('D',
  'you for a million years To make you feel my love',
  '5afddfd32a1a05fe393d96b7'),
 ('D',
  'million years To make you feel my love I know you',
 

In [161]:
def make_phrase_is_minor_list(chord_phrase_tuples):

    phrases = []
    chords = []
    is_minor = []
    song_ids = []

    for chord_phrase_tup in chord_phrase_tuples:
        phrases.append(chord_phrase_tup[1])
        chords.append(chord_phrase_tup[0])
        is_minor.append('m' in chord_phrase_tup[0])
        song_ids.append(chord_phrase_tup[2])

    return phrases, chords, is_minor, song_ids

good_chords = ['A', 'C', 'D', 'G']

def get_list_of_chords(chord_phrase_tuples):
    is_A = []
    is_C = []
    is_D = []
    is_G = []
    other_chord = []
    for tup in chord_phrase_tuples:
        is_A.append(tup[0] == 'A')
        is_C.append(tup[0] == 'C')
        is_D.append(tup[0] == 'D')
        is_G.append(tup[0] == 'G')
        other_chord.append(tup[0] not in good_chords)
    return is_A, is_C, is_D, is_G, other_chord

def make_phrase_chord_df(phrases, chords, is_minor, is_A, is_C, is_D, is_G, other_chord):
    df = pd.DataFrame({
        'chords': chords,
        'A': is_A,
        'C': is_C,
        'D': is_D,
        'G': is_G,
        'other_chord': other_chord,
        'is_minor': is_minor,
        'words': phrases
    })
    return df


In [53]:
is_A, is_C, is_D, is_G, other_chord = get_list_of_chords(pct)

In [163]:
phrases, chords, is_minor, song_ids = make_phrase_is_minor_list(pct)

In [164]:
song_ids

['5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd32a1a05fe393d96b7',
 '5afddfd42a1a05fe393d96b9',
 '5afddfd42a1a05fe393d96b9',
 '5afddfd42a1a

In [57]:
make_phrase_chord_df(phrases, chords, is_minor, is_A, is_C, is_D, is_G, other_chord)

,A,C,D,G,chord,is_minor,other_chord,words
0,False,False,False,True,G,False,False,
1,False,True,False,False,C,False,False,
2,False,True,False,False,C,False,False,to the doctor he says I'm alright I knew he was
3,False,True,False,False,C,False,False,knew he was lyin' I'm losing my sight He shoul...
4,False,False,False,True,G,False,False,should have examined the eyes of my mind 20 20...
5,False,False,True,False,D,False,False,eyes of my mind 20 20 vision and walkin' round...
6,False,False,False,False,E,False,True,
7,True,False,False,False,A,False,False,
8,False,False,False,False,E,False,True,
9,True,False,False,False,A,False,False,


In [166]:
def make_phrase_is_minor_list(chord_phrase_tuples):

    phrases = []
    chords = []
    is_minor = []
    song_ids = []

    for chord_phrase_tup in chord_phrase_tuples:
        phrases.append(chord_phrase_tup[1])
        chords.append(chord_phrase_tup[0])
        is_minor.append('m' in chord_phrase_tup[0])
        song_ids.append(chord_phrase_tup[2])

    return phrases, chords, is_minor, song_ids


good_chords = ['A','B','C','D','E','F','G','G#','A7','D7','Em','Am','Bm','Dm','Bb']

def get_list_of_chords(chord_phrase_tuples):
    is_A = []
    is_B = []
    is_C = []
    is_D = []
    is_E = []
    is_F = []
    is_G = []
    is_A7 = []
    is_D7 = []
    is_Em = []
    is_Am = []
    is_Bm = []
    is_Dm = []
    is_Bb = []
    other_chord = []
    for tup in chord_phrase_tuples:
        is_A.append(tup[0] == 'A')
        is_B.append(tup[0] == 'B')
        is_C.append(tup[0] == 'C')
        is_D.append(tup[0] == 'D')
        is_E.append(tup[0] == 'E')
        is_F.append(tup[0] == 'F')
        is_G.append(tup[0] == 'G' or tup[0] == 'G#')
        is_A7.append(tup[0] == 'A7')
        is_D7.append(tup[0] == 'D7')
        is_Em.append(tup[0] == 'Em')
        is_Am.append(tup[0] == 'Am')
        is_Bm.append(tup[0] == 'Bm')
        is_Dm.append(tup[0] == 'Dm')
        is_Bb.append(tup[0] == 'Bb')
        other_chord.append(tup[0] not in good_chords)
    return is_A, is_B, is_C, is_D, is_E, is_F, is_G, is_A7, is_D7, is_Em, is_Am, is_Bm, is_Bb, is_Dm, other_chord


def make_phrase_chord_df(chord_phrase_tuples):

    phrases, chords, is_minor, song_ids = make_phrase_is_minor_list(chord_phrase_tuples)

    is_A, is_B, is_C, is_D, is_E, is_F, is_G, is_A7, is_D7, is_Em, is_Am, is_Bm, is_Bb, is_Dm, other_chord = get_list_of_chords(chord_phrase_tuples)

    df = pd.DataFrame({
        'song_id': song_ids,
        'chord': chords,
        'A': is_A,
        'B': is_B,
        'C': is_C,
        'D': is_D,
        'E': is_E,
        'F': is_F,
        'G': is_G,
        'A7': is_A7,
        'D7': is_D7,
        'Em': is_Em,
        'Am': is_Am,
        'Bm': is_Bm,
        'Dm': is_Dm,
        'Bb': is_Bb,
        'other_chord': other_chord,
        'is_minor': is_minor,
        'words': phrases
    })

    return df

In [167]:
pct_more = get_all_phrase_chord_tuples(parsed_songs[1111:1357])
pct_more

[('D', '', '5afddfe42a1a05fe393d9c58'),
 ('Am7', '', '5afddfe42a1a05fe393d9c58'),
 ('D',
  'Well I lived with a child of snow When I was',
  '5afddfe42a1a05fe393d9c58'),
 ('D',
  'of snow When I was a soldier And I fought every',
  '5afddfe42a1a05fe393d9c58'),
 ('Am7',
  'I was a soldier And I fought every man for her',
  '5afddfe42a1a05fe393d9c58'),
 ('D',
  'soldier And I fought every man for her Until the nights',
  '5afddfe42a1a05fe393d9c58'),
 ('D', '', '5afddfe42a1a05fe393d9c5a'),
 ('G', '', '5afddfe42a1a05fe393d9c5a'),
 ('D',
  'song I ever had You got me singing ever since the',
  '5afddfe42a1a05fe393d9c5a'),
 ('G',
  'had You got me singing ever since the river died you',
  '5afddfe42a1a05fe393d9c5a'),
 ('D',
  'since the river died you got me thinking of the places',
  '5afddfe42a1a05fe393d9c5a'),
 ('Bm',
  'places we could hide You got me singing even though the',
  '5afddfe42a1a05fe393d9c5a'),
 ('m',
  'You got me singing even though the world is gone you',
  '5afddfe42a1a0

In [168]:
df_c = make_phrase_chord_df(pct_more)
df_c

,A,A7,Am,B,Bb,Bm,C,D,D7,Dm,E,Em,F,G,chord,is_minor,other_chord,song_id,words
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,D,False,False,5afddfe42a1a05fe393d9c58,
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Am7,True,True,5afddfe42a1a05fe393d9c58,
2,False,False,False,False,False,False,False,True,False,False,False,False,False,False,D,False,False,5afddfe42a1a05fe393d9c58,Well I lived with a child of snow When I was
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,D,False,False,5afddfe42a1a05fe393d9c58,of snow When I was a soldier And I fought every
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Am7,True,True,5afddfe42a1a05fe393d9c58,I was a soldier And I fought every man for her
5,False,False,False,False,False,False,False,True,False,False,False,False,False,False,D,False,False,5afddfe42a1a05fe393d9c58,soldier And I fought every man for her Until t...
6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,D,False,False,5afddfe42a1a05fe393d9c5a,
7,False,False,False,False,False,False,False,False,False,False,False,False,False,True,G,False,False,5afddfe42a1a05fe393d9c5a,
8,False,False,False,False,False,False,False,True,False,False,False,False,False,False,D,False,False,5afddfe42a1a05fe393d9c5a,song I ever had You got me singing ever since the
9,False,False,False,False,False,False,False,False,False,False,False,False,False,True,G,False,False,5afddfe42a1a05fe393d9c5a,had You got me singing ever since the river di...


In [106]:
df_c['chord'].value_counts()

G            2795
D            2061
C            1832
A            1361
F            1226
Em            680
Am            654
E             479
Bm            391
m             254
B             225
D7            212
Em7           177
E7            170
A7            141
Bb            133
B7            121
Dm            104
G7             83
Am7            66
Gm             58
Asus4          40
Dsus2          39
Dmaj7          30
C7             29
Dsus4          28
7              27
D7sus4         25
Cm             24
Bm7            21
             ... 
mother          1
2               1
Cmaj7           1
tab             1
gleaming        1
pause           1
8               1
intro           1
sweet           1
Welcome         1
one             1
02220           1
Bbmaj           1
precious        1
Then            1
N               1
shore           1
Amaj            1
Dadd2           1
caress          1
4               1
sensed          1
earthmade       1
C9              1
try       

In [215]:
tr, te = train_test_split(df_c['song_id'])

In [234]:
len(df_c.song_id)

13944

In [235]:
len(set(df_c.song_id))

245

In [216]:
df_train = df_c[df_c.song_id.isin(tr)]

In [217]:
df_test = df_c[df_c.song_id.isin(te)]

In [233]:
set(tr) & set(te)

{'5afddfe42a1a05fe393d9c58',
 '5afddfe42a1a05fe393d9c5a',
 '5afddfe42a1a05fe393d9c5c',
 '5afddfe42a1a05fe393d9c5e',
 '5afddfe42a1a05fe393d9c60',
 '5afddfe42a1a05fe393d9c62',
 '5afddfe42a1a05fe393d9c64',
 '5afddfe42a1a05fe393d9c66',
 '5afddfe42a1a05fe393d9c68',
 '5afddfe42a1a05fe393d9c6a',
 '5afddfe42a1a05fe393d9c6c',
 '5afddfe42a1a05fe393d9c6e',
 '5afddfe42a1a05fe393d9c70',
 '5afddfe42a1a05fe393d9c72',
 '5afddfe42a1a05fe393d9c74',
 '5afddfe42a1a05fe393d9c76',
 '5afddfe42a1a05fe393d9c78',
 '5afddfe42a1a05fe393d9c7c',
 '5afddfe42a1a05fe393d9c7e',
 '5afddfe42a1a05fe393d9c80',
 '5afddfe42a1a05fe393d9c82',
 '5afddfe42a1a05fe393d9c84',
 '5afddfe42a1a05fe393d9c86',
 '5afddfe42a1a05fe393d9c88',
 '5afddfe42a1a05fe393d9c8a',
 '5afddfe52a1a05fe393d9c8c',
 '5afddfe52a1a05fe393d9c8e',
 '5afddfe52a1a05fe393d9c90',
 '5afddfe52a1a05fe393d9c93',
 '5afddfe52a1a05fe393d9c95',
 '5afddfe52a1a05fe393d9c97',
 '5afddfe52a1a05fe393d9c99',
 '5afddfe52a1a05fe393d9c9b',
 '5afddfe52a1a05fe393d9c9d',
 '5afddfe52a1a

In [228]:
len(tr)

10458

In [218]:
df_test.shape

(13938, 19)

In [219]:
df_c.shape

(13944, 19)

In [220]:
df_train.shape

(13944, 19)

In [221]:
df_c['chord'].values

array(['D', 'Am7', 'D', ..., 'F', 'C', 'C'], dtype=object)

In [224]:
tfidf = TfidfVectorizer()
logistic = LogisticRegression()

In [225]:
chord_list = ['A','B','C','D','E','F','G','G#','A7','D7','Em','Am','Bm','Dm','Bb']

chord_prob_dict = {}

for chord in chord_list:
    
    X_train = df_train['words']
    y_train = df_train[chord]
    X_test = df_test['words']
    y_test = df_test[chord]
    tfidf.fit(X_train)
    tr_matrix = tfidf.transform(X_train)
    logistic.fit(X_train, y_train)
    te_matrix = tfidf.transform(X_test)
    y_hat_log = logistic.predict_proba(X_test)
    
    chord_prob_dict[chord] = y_hat_log[0][1]
    
best_chord = max(chord_prob_dict.items(), key=lambda k: k[1])
    
return best_chord 


ValueError: could not convert string to float: 'right on me You put the load right on me'